In [34]:
#Create connection with MYSQL
import mysql.connector
from mysql.connector import errorcode

#import jason library
import json

#import pandas library for working with dataframes
import pandas as pd

#import datetime module for dates manipulation
from datetime import datetime

In [35]:
def get_config():
    config = {} #setup dictionary
    jfile = open('config.json')
    config = json.load(jfile)
    
    return config

In [36]:
config = get_config()

In [37]:
def setup_connection(config):
    '''
        This function is to set up connection with MYSQL 'edw' database.
        And define a cursor for records processing.|
    '''

    try:
        cnx = mysql.connector.connect(**config)
    
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print('**ERR: Invalid id or password**')
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print(err.errno)
                print('**ERR: Wrong dbname or db does not exists**')
        else:
                print(err)
    else:
            print('**MSG: Connection to db successful**')
            cursor = cnx.cursor()
            return cnx, cursor

In [38]:
def getcsv():
    '''
        Read the Legacy customer data from customer csv file and build a dataframe.
    '''
    print('**MSG: Reading customer csv data**')
    trandata = pd.read_csv('additional_transactions.csv', index_col=False, delimiter=',', parse_dates=['date'])
    print(trandata.head(2))
    return trandata

In [39]:
cnx, cursor = setup_connection(config)

**MSG: Connection to db successful**


In [30]:
tran_df = getcsv()

**MSG: Reading customer csv data**
                 date             email        txn_id        item_id  qty  \
0 2021-12-08 14:23:33   KCrum@gmail.com  T-8O1GQQY3I9   CADETBLUE-1P    4   
1 2021-12-06 05:17:55  dsmith@yahoo.com  T-047QQ8F3QG  SANDYBROWN-2Q    3   

         color        size  price  
0   Cadet Blue      1 Pint  12.80  
1  Sandy Brown  1/2 Gallon  34.56  


In [40]:
def insert_tran(cnx, cursor, trandata):

    print('**MSG: Inserting data to customer_stg**')
    
    # count total rows inserted
    rcnt = 0          
    
    for i,row in trandata.iterrows():

        add_tran = ("INSERT INTO `edw`.`transaction_stg` "
                   "(`tranid`, `customer_email`, `order_date`, `line_num`, `product`, `quantity`, `color`, `item_size`, `total_price`)"
                   "VALUES (%(tranid)s, %(customer_email)s, %(order_date)s, %(line_num)s, %(product)s, %(quantity)s, %(color)s, %(item_size)s, %(total_price)s)"
                )
        try:
                data_tran = {"tranid": row.txn_id, "customer_email": row.email, "order_date": row.date,
                             "line_num": 1, "product": row.item_id, "quantity": row.qty, 
                             "color": row.color, "item_size": row.size, 
                             "total_price": row.price
                            }
                cursor.execute(add_tran,data_tran)
                rcnt = rcnt + 1
                prv_tran = trandata.txn_id
        except mysql.connector.Error as e:
                print('**ERR: Error inserting rows: ', e)
                cnx.commit()
                cursor.close()
                cnx.close()
    
    print('**MSG: Total Rows inserted to customer_stg: ', rcnt)

In [41]:
insert_tran(cnx,cursor,tran_df)

**MSG: Inserting data to customer_stg**
**MSG: Total Rows inserted to customer_stg:  236


In [42]:
cnx.commit()
cursor.close()
cnx.close()